![Logo de AA1](logo_AA1_texto_small.png) 
# Sesión 11 - Evaluación del rendimiento tras buscar hiperparámetros

En la sesión de prácticas número 9 veíamos que para conocer el rendimiento que tendrá un modelo en producción es necesario evaluarlo con casos no vistos durante el entrenamiento.

En la sesión de prácticas número 10 vimos cómo buscar los mejores hiperparámetros para una determinado algoritmo y para ello hemos utilizado todos los ejemplos que teníamos disponibles, así que el rendimiento que obtuvimos puede no ser el que tenga el modelo en producción.

En esta sesión vamos a ver cómo obtener el rendimiento del modelo en producción a la vez que se buscan los mejores hiperparámetros.

## 11.1 Método 1: GridSearchCV o RandomizedSearchCV dentro de una validación cruzada

Como ya hemos indicado en otras ocasiones, la manera de ver qué rendimiento va a tener nuestro sistema en producción es probándolo ante casos no vistos, así que tendremos que simular esa situación.

La mejor forma es mediante una validación cruzada:

![Validación cruzada](fig_cv.png)

Pero ahora, en cada iteración de la validación cruzada debemos realizar una búsqueda de hiperparámetros, ya sea utilizando `GridSearchCV()` o `RandomizedSearchCV()`. **¡Esto supone anidar dos validaciones cruzadas!**

Veamos cómo implementarlo utilizando el conjunto que ya hemos utilizado en sesiones anteriores. Primero lo cargamos, lo preprocesamos y creamos una instancia del `KNN`:

In [1]:
# se importan las librerías
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, ParameterGrid, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import randint

# se llama a la función read_csv
# no tiene missing y las columnas están separadas por ','
# tampoco cabecera, así que hay que dar nombre a las columnas (como en el names no vienen indicados creamos nombres)
cabecera = ['atr'+str(x) for x in range(1,35)]
cabecera.append('clase')
df = pd.read_csv('ionosphere.data', names=cabecera)
filas, columnas = df.shape


# separamos los atributos y los almacenamos en X
X = df.drop(['clase'], axis=1)
display(X)

class_enc = preprocessing.LabelEncoder()
df['clase'] = class_enc.fit_transform(df['clase'])

# separamos la clase y la almacenamos en Y
y = df['clase']
display(y)

print('\n##########################################')
print('### K-vecinos')
print('##########################################')
# creamos una instancia del KNN
knn_sis = KNeighborsClassifier()

,atr1,atr2,atr3,atr4,atr5,atr6,atr7,atr8,atr9,atr10,...,atr25,atr26,atr27,atr28,atr29,atr30,atr31,atr32,atr33,atr34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,0.95378,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.94520,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.93988,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,0.91050,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243


0      1
1      0
2      1
3      0
4      1
      ..
346    1
347    1
348    1
349    1
350    1
Name: clase, Length: 351, dtype: int64


##########################################
### K-vecinos
##########################################


Posteriormente definimos el espacio de búsqueda para los hiperparámetros y creamos una `GridSeachCV()`, que bien podría haber sido una `RandomizedSearchCV()`:

In [2]:
# se definen los valores de los hiperparámetros que se quieren  probar
weights = ['uniform', 'distance'] # weights : {'uniform', 'distance'}
p = [1, 2, 3] # p : int, default=2 => Euclídea
n_neighbors = [1, 2, 3, 4, 5]

# y se introducen en un diccionario
hyperparameters = dict(weights=weights, p=p, n_neighbors=n_neighbors)

display(hyperparameters)

# se crea un generador de folds estratificados partiendo el conjunto en 5 trozos
folds5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

# creamos una grid search para el KNN donde le pasamos los hiperparámetros que queremos probar
gs = GridSearchCV(knn_sis, hyperparameters, scoring='accuracy', cv=folds5, verbose=1, n_jobs=-1)

{'weights': ['uniform', 'distance'],
 'p': [1, 2, 3],
 'n_neighbors': [1, 2, 3, 4, 5]}

Finalmente, ejecutamos la función `cross_val_score()` que efectuará la validación cruzada que deseemos. En este caso vamos a utilizar 10 folds, por tanto, el número de entrenamientos será $2 \times 3 \times 5 \times 5 \times 10 = 1500$ (2 valores para `weights`, 3 valores para `p`, 5 valores para `n_neighbors`, la grid search tiene 5 folds y la validación cruzada 10 folds).

Para que no se nos llene la consola de ejecuciones es recomendable bajar el `verbose` a 1 tanto en `GridSearchCV()` como en `cross_val_score()`:

In [6]:
# se crea un generador de folds estratificados partiendo el conjunto en 10 trozos
folds10 = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

# se realiza la validación cruzada para el KNN
scores_knn = cross_val_score(gs, X, y, cv=folds10, scoring='accuracy', verbose=1)
print("KNN (mean+-std): %0.4f +- %0.4f" % (scores_knn.mean(), scores_knn.std()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
KNN (mean+-std): 0.9117 +- 0.0518


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.1s finished


Como vemos, el tiempo de ejecución puede comenzar a elevarse peligrosamente, así que cuando necesitemos utilizar esta técnica debemos ser conscientes del número de ejecuciones que implica del tiempo que puede llegar a tardar.

Un inconveniente que se presenta es el hecho de que en cada iteración de la validación cruzada puede resultar una combinación de hiperparámetros diferentes, así que aunque hemos sido capaces de obtener el rendimiento en producción, no sabemos cuáles son los hiperparámetros que debemos utilizar. Esto se solucionaría realizando una `GridSearchCV()` como hacíamos en la práctica anterior y así tendríamos los mejores hiperparámetros.

## 11.2 Método 2: Entrenamiento - validación - test

Cuando anidar dos validaciones cruzadas suponga que el tiempo de ejecución es demasiado elevado, entonces podemos recurrir a este método que como veremos es menos fiable, pero mucho menos costoso computacionalmente. Consiste en dividir el conjunto de datos varias veces:

![Train-val-test](fig_train_val_test.png) 

Veamos cómo se procede:
1. Primero se separa en un conjunto para entrenar (training set) y en otro para evaluar (test set). El conjunto de test no podremos utilizarlo durante la búsqueda de hiperparámetros.
2. Así que para buscar los hiperparámetros necesitamos hacer otra división, pero esta vez del conjunto de entrenamiento: tendremos un conjunto para entrenar las diferentes combinaciones de hiperparámetros y las probaremos sobre el conjunto de validación que suele llamarse *validation set* o *development set*. 
3. Una vez que se ha encontrado la mejor combinación de hiperparámetros habrá que entrenar el sistema utilizando el conjunto de entrenmaiento completo y evaluar sobre el conjunto de test para así tener una aproximación al rendimiento que el modelo tendrá ante casos no vistos.

Veamos cómo implementarlo:

In [7]:
# separamos los datos en training set y test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1234, stratify=y)

# separamos los datos de entrenamiento en train y val para la búsqueda de hiperparámetros
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1234, stratify=y_train)

print("\nNúmero de ejemplos inicial")
print("X:", X.shape[0])

print("\nNúmero de ejemplos para train y test")
print("X_train:", X_train.shape[0])
print("X_test:", X_test.shape[0])

print("\nNúmero de ejemplos tras dividir nuevamente el train")
print("X_tr:", X_tr.shape[0])
print("X_val:", X_val.shape[0])


Número de ejemplos inicial
X: 351

Número de ejemplos para train y test
X_train: 263
X_test: 88

Número de ejemplos tras dividir nuevamente el train
X_tr: 197
X_val: 66


Lo primero que hacemos es separar, utilizando `train_test_split()`, el conjunto de datos en los conjuntos *train* y *test* (esta función ya realiza un barajado de los ejemplos antes de partir). El conjunto de test quedará reservado para la evaluación final actuando como conjunto de casos no vistos, así que el conjunto de train es el que utilizaremos para la búsqueda de hiperparámetros.

Por tanto, debemos volver a dividirlo para tener un subconjunto con el que entrenar (`tr`) y otro con el que evaluar o validar el rendimiento (`val`).

En ambas divisiones hemos separado un 25% de los ejemplos para test y validación. Este porcentaje dependerá de cada conjunto de datos que estemos tratando. Lo deseable es siempre tener conjuntos de tests que sean representativos, así que cuanto mayor sea mejor, sin embargo, si tenemos pocos datos en el conjunto, reservar muchos datos para test puede ser contraproducente puesto que no estaríamos utilizando esos datos para entrenar el modelo.

Ese es el problema de esta forma de buscar hiperparámetros.

Si nos fijamos en la salida del código anterior, contamos inicialmente con 351 ejemplos y hemos apartado 88 para la evaluación final ¿son representativos? Esperamos que sí. Para buscar hyperparámetros vamos a utulizar los 263 ejemplos de conjunto de entrenamiento que dividimos nuevamente quedando 66 ejemplos para la validación, ¿son representativos? Volvemos a esperar que sí. Si os dáis cuenta, estamos fiando la búsqueda de hiperparámetros al rendimiento que tengan los modelos sobre 66 ejemplos. Puede salir bien o puede salir mal, depende de si son representativos o no.

Una vez hechas las divisiones se preparan las combinaciones de hiperparámetros:

In [8]:
# se definen los valores de los hiperparámetros que se quieren  probar
weights = ['uniform', 'distance'] # weights : {'uniform', 'distance'}
p = [1, 2, 3] # p : int, default=2 => Euclídea
n_neighbors = [1, 2, 3, 4, 5]

# y se introducen en un diccionario
hyperparameters = dict(weights=weights, p=p, n_neighbors=n_neighbors)

display(hyperparameters)

# creamos un grid con los hiperparámetros
grid = ParameterGrid(hyperparameters)

{'weights': ['uniform', 'distance'],
 'p': [1, 2, 3],
 'n_neighbors': [1, 2, 3, 4, 5]}

En esencia se hace lo mismo que habíamos hecho para el `GridSearchCV()`, la única diferencia es la última línea en la que utilizamos la función `ParameterGrid()` para que genere todas las combinaciones posible de hiperparámetros con los valores que le hemos dado.

Esas combinaciones las utilizaremos dentro de un bucle, como vemos en el siguiente trozo de código:

In [9]:
# inicializamos variables para almacenar los mejores
best_acc = 0
best_hyperparams = None

# hacemos un bucle para probar todas las combinaciones
for hyperparams in grid:
    knn_sis.set_params(**hyperparams)      # asignamos los hyperparámetros
    print(knn_sis)
    knn_sis.fit(X_tr, y_tr)                 # entrenamos con X_tr
    y_val_pred = knn_sis.predict(X_val)     # evaluamos con X_val
    acc = accuracy_score(y_val, y_val_pred) # calculamos accuracy y_val vs y_val_pred
    if acc > best_acc:
        best_acc = acc
        best_hyperparams = hyperparams

print("Mejor combinación de hiperparámetros:", best_hyperparams)
print("Mejor rendimiento obtenido: %.4f" % best_acc)

KNeighborsClassifier(n_neighbors=1, p=1)
KNeighborsClassifier(n_neighbors=1, p=1, weights='distance')
KNeighborsClassifier(n_neighbors=1)
KNeighborsClassifier(n_neighbors=1, weights='distance')
KNeighborsClassifier(n_neighbors=1, p=3)
KNeighborsClassifier(n_neighbors=1, p=3, weights='distance')
KNeighborsClassifier(n_neighbors=2, p=1)
KNeighborsClassifier(n_neighbors=2, p=1, weights='distance')
KNeighborsClassifier(n_neighbors=2)
KNeighborsClassifier(n_neighbors=2, weights='distance')
KNeighborsClassifier(n_neighbors=2, p=3)
KNeighborsClassifier(n_neighbors=2, p=3, weights='distance')
KNeighborsClassifier(n_neighbors=3, p=1)
KNeighborsClassifier(n_neighbors=3, p=1, weights='distance')
KNeighborsClassifier(n_neighbors=3)
KNeighborsClassifier(n_neighbors=3, weights='distance')
KNeighborsClassifier(n_neighbors=3, p=3)
KNeighborsClassifier(n_neighbors=3, p=3, weights='distance')
KNeighborsClassifier(n_neighbors=4, p=1)
KNeighborsClassifier(n_neighbors=4, p=1, weights='distance')
KNeighbors

Cuando queremos pasarle varios hiperparámetros a un sistema utilizando `set_params()` debemos pasárselos como argumento en un diccionario con la sintaxis `**diccionario`.

Más detalles en: https://stackoverflow.com/questions/36901/what-does-double-star-asterisk-and-star-asterisk-do-for-parameters

En cada iteración se asigna al sistema la combinación de hiperparámetros correspondiente y se entrena utilizando la partición de datos de entrenamiento (para la búsqueda) y se evalúa sobre la partición de validación.

Si se obtiene un rendimiento mejor, entonces se almacena la accuracy y la combinación de hiperparámetros.

Al finalizar el bucle ya tendremos la mejor combinación de hiperparámetros y, como se ve en el código siguiente, ya solo nos queda entrenar el modelo con el conjunto de entrenamiento al completo utilizando la mejor combinación de hyperparámetros y evaluar sobre el conjunto de test:

In [10]:
# asignamos los mejores hiperparámetros
best_model = knn_sis.set_params(**best_hyperparams)
print(best_model)

# reentrenamos el vecino más próximo
best_model.fit(X_train, y_train)            # se entrena sobre la partición de train
y_test_pred = best_model.predict(X_test)    # se predice sobre la partición de test
print("Accuracy sobre casos no vistos: %.4f" % accuracy_score(y_test, y_test_pred))

KNeighborsClassifier(n_neighbors=2, p=1)
Accuracy sobre casos no vistos: 0.9205


## 11.3 Mezcla de los dos métodos

También es posible realizar una combinación de los dos métodos expuestos anteriormente.

Podríamos hacer un hold-out 70-30 y realizar una búsqueda de hiperparámetros sobre el conjunto de entrenamiento utilizando `GridSearchCV` o `RandomizedSearchCV`. 

Siguiendo este procedimiento eliminaríamos el exceso de tiempo que supone la validación cruzada.

## Ejercicios

1. Carga el fichero **heart_failure_clinical_records_dataset.csv** (es un archivo de texto). 
2. Utiliza los dos métodos vistos en esta sesión para calcular el rendimiento del sistema tras la búsqueda de hiperparámetros.

**OJO**, ten en cuenta que los atributos tienen escalas diferentes, así que deberás crear un pipeline.

Estos ejercicios no es necesario entregarlos.